In [3]:
from src.model.training import tune_train
from src.utils.utils import get_env_configs, get_policy_configs, get_model_configs
from src.utils.Params import Params
import ray


if __name__ == "__main__":
    params = Params()
    ray.init(local_mode=params.debug)

    # Get configs
    env_config = get_env_configs(params)
    policy_configs = get_policy_configs(params)
    model_configs = get_model_configs(params)

    configs = {
        "env": params.env_name,
        "env_config": env_config,
        "framework": params.framework,
        "num_workers": params.num_workers,
        "num_gpus": params.num_gpus,
        "batch_mode": "complete_episodes",
        #"train_batch_size": 400,
        "rollout_fragment_length": 50,

        # PPO parameter
        "lr": 3e-4,
        "lambda": .95,
        "gamma": .998,
        "entropy_coeff": 0.01,
        "clip_param": 0.2,
        "use_critic": True,
        "use_gae": True,
        "grad_clip": 5,
        "num_sgd_iter": 10,

        # Callbacks
        "callbacks": {},

        # Model
        "model": model_configs,

        # Multiagent
        "multiagent": policy_configs
    }

    ## TRAIING
    #visual_train(params, configs)
    tune_train(params, configs)


2021-10-19 16:51:45,509	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc
PPO_collab_nav_14e82_13db1dff,RUNNING,


(pid=268994) 2021-10-19 16:51:48,010	INFO trainer.py:710 -- Executing eagerly, with eager_tracing=False
(pid=268994) 2021-10-19 16:51:48,010	WARNING ppo.py:135 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=15 num_envs_per_worker=1 rollout_fragment_length=50)! Auto-adjusting `rollout_fragment_length` to 266.
(pid=268994) 2021-10-19 16:51:48,010	INFO trainer.py:726 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=268986) 2021-10-19 16:51:51,086	WARNING deprecation.py:38 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, **kwargs)` instead. This will raise an error in the future!
(pid=268994) 2021-10-19 16:51:52,542	WARNING deprecation.py:38 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
(pid=268994) 2021-10-19 16:51:52,542	WARNING deprecation.py:38 -- Deprec

Result for PPO_collab_nav_14e82_13db1dff:
  agent_timesteps_total: 6000
  custom_metrics: {}
  date: 2021-10-19_16-51-58
  done: false
  episode_len_mean: 200.0
  episode_media: {}
  episode_reward_max: -6330.8
  episode_reward_mean: -19862.92
  episode_reward_min: -36240.0
  episodes_this_iter: 30
  episodes_total: 30
  experiment_id: 04c06d955b354b9085da1832ef70340f
  hostname: Ublion20
  info:
    learner:
      agent_0:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.0003000000142492354
          entropy: 1.6044412851333618
          entropy_coeff: 0.009999999776482582
          kl: 0.005054359324276447
          policy_loss: -0.0035191585775464773
          total_loss: 3792749.5
          vf_explained_var: 0.002354373224079609
          vf_loss: 3792749.5
        model: {}
      agent_1:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.0003000000142492

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_collab_nav_14e82_13db1dff,RUNNING,192.168.178.20:268994,1,7.29112,6000,-19862.9,-6330.8,-36240,200


...

Result for PPO_collab_nav_14e82_13db1dff:
  agent_timesteps_total: 18000
  custom_metrics: {}
  date: 2021-10-19_16-52-12
  done: false
  episode_len_mean: 200.0
  episode_media: {}
  episode_reward_max: -6330.8
  episode_reward_mean: -41234.21333333333
  episode_reward_min: -109515.20000000001
  episodes_this_iter: 30
  episodes_total: 90
  experiment_id: 04c06d955b354b9085da1832ef70340f
  hostname: Ublion20
  info:
    learner:
      agent_0:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.0003000000142492354
          entropy: 1.580891728401184
          entropy_coeff: 0.009999999776482582
          kl: 0.01347193494439125
          policy_loss: 0.00940072350203991
          total_loss: 27039316.0
          vf_explained_var: 0.00024166007642634213
          vf_loss: 27039318.0
        model: {}
      agent_1:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_collab_nav_14e82_13db1dff,RUNNING,192.168.178.20:268994,3,21.5888,18000,-41234.2,-6330.8,-109515,200


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_collab_nav_14e82_13db1dff,RUNNING,192.168.178.20:268994,3,21.5888,18000,-41234.2,-6330.8,-109515,200


(pid=268994) 2021-10-19 16:52:12,801	ERROR worker.py:428 -- SystemExit was raised from the worker
(pid=268994) Traceback (most recent call last):
(pid=268994)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.task_execution_handler
(pid=268994)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=268994)   File "python/ray/_raylet.pyx", line 525, in ray._raylet.execute_task
(pid=268994)   File "python/ray/_raylet.pyx", line 532, in ray._raylet.execute_task
(pid=268994)   File "python/ray/_raylet.pyx", line 536, in ray._raylet.execute_task
(pid=268994)   File "python/ray/_raylet.pyx", line 486, in ray._raylet.execute_task.function_executor
(pid=268994)   File "/home/dizzi/anaconda3/envs/MAMBRL/lib/python3.9/site-packages/ray/_private/function_manager.py", line 563, in actor_method_executor
(pid=268994)     return method(__ray_actor, *args, **kwargs)
(pid=268994)   File "/home/dizzi/anaconda3/envs/MAMBRL/lib/python3.9/site-packages/ray/tune/trainable.py", l

2021-10-19 16:52:12,971	INFO tune.py:561 -- Total run time: 26.69 seconds (26.42 seconds for the tuning loop).
2021-10-19 16:52:12,973	WARNING tune.py:565 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
